# Problem
Our project was to train our computer to play Tetris. We used an implementation of Tetris found online (source in** References**). To train our bot, we decided to use DEAP and a genetic algorithm. We had some challenges when starting this project. First, we had to adapt the human-player game to accept a computer input. We also had to decide what heuristics to evaluate potential moves and select one. Lastly, we had to train the computer and tweak the genetic algorithm parameters

# Methodology

### Interface between computer and Tetris
We defined a Brain class that takes in a board state (represented by the Data class) and enumerates all potential rotations and translations, followed by a DROP action. Then, each resulting board state is scored according to weighted heuristics (**Heuristics** specified below). What this meant is that given characteristics A, B, and C, the score of a given board state would be a\*A + b\*B + c\*C where a, b, and c are real weights that reflect how beneficial or harmful the corresponding characteristic is to game performance. For example, a simple heuristic would be number of filled rows, and a logical corresponding weight would likely be fairly high and positive, since we want to choose moves that result in boards with filled rows. Once each board state is evaluated by these weighted heuristics, we chose the highest score and returned the combination of moves that led to that state.

The Tetris game was modified to have an additional *run_brain* function to support computer-play, while we kept the original *run* function to support human-play. The *run_brain* function takes in 4 weights (to reflect the 4 heuristics chosen below) and creates a Brain with the given weights. Every iteration of the game loop, the brain is given the new board state and asked for a list of moves to execute, which are then executed. We had 2 modes for *run_brain*: one for training, and one for demonstration. The training mode ran without the pygame update code, so that training could be uninterrupted by the overhead for graphics. Additionally we modified the complex scoring system of the old game to a simpler one that awarded 1 point for placing a piece and 10 points for clearing a row.

### Heuristics for choosing an optimal move
We used a helpful blog article (source in **References**) which specified 4 basic heuristics:
1. Aggregate height
2. Complete lines
3. Number of holes
4. Bumpiness

<img src='heuristic.png'>

The height of a column is the height of the top block in each column (so empty spaces are also counted). In the diagram above, this would be the number above each column. **Aggregate height** would then be the sum of each column's height. For the diagram above, this value would be 3+5+5+5+6+6+5+4+4+5 = **48**. This is a heuristic that should probably be minimized, since once the blocks reach the top the game is over. Thus, we expect a negative weight for this value.

The number of **complete lines** is the number of rows that are about to be cleared. For the diagram above, this would be **2**. This is a very intuitive value to maximize, so we expected a positive weight.

The **number of holes** is the number of empty locations that have blocks above. For anyone who has played Tetris before, holes are not good typically, since they prevent the row they are in from being cleared, and they cannot be filled immediately. For the diagram above, this value would be **2**. This weight was expected to be negative.

Lastly, **bumpiness** is the sum of the differences between adjacent column heights. A high bumpiness means that neighboring columns vary widely in height. This oftens means the presence of rows that could and should be cleared, so we also expected this heuristic to be penalized and have a negative weight. For the diagram above, this value would be $$|3-5| + |5-5| + ... + |4-4| + |4-5| = \textbf{6}$$

In our Brain class, we had a method to calculate each of these heuristics given a board state. 

### Training
To train, we had standard DEAP genetic algorithm code with crossover and mutation functions for real numbers. Each individual was a randomly generated string of 4 weights. For each generation, we looped through each individual and ran *run_brain* with the individual as the weights. The function would return a score which we assigned as the fitness for that individual. Then, we would generate the next generation using canonical GA methods.

During training, statistics on best, average, and standard deviation of scores were collected (see **Results** below). Once complete, the best string of 4 weights was written to *best_weights.txt*. 

# Results
The final weights we used were $$\textbf{[0.06346301632540774, 0.2653280714475912, -1.2246601893643692, -0.27134621659306046]}$$

The weight for **Aggregate height** is interesting, since a value of 0.063 actually rewards higher aggregate height. The rest of the weights fall according to our prediction though.

When training, we varied the number of generations and found that training past 12 generations seemed to contribute little to increasing average score, so we ended training after 12 generations. Additionally, since a full game of Tetris is played for each individual, the generations take increasing time to run as the weights get more "smart". We wanted to keep number of generations on the lower side.

### Statistics for run 1 (n_gen = 10)
<img src='run1.png'>

Since average score seemed to still increase, we decided to increase the number of generations to 15.

### Statistics for run 2 (n_gen = 15)
<img src='run2.png'>

The average score seemed to plateau past 12 generations, so we lowered n_gen to 12. Additionally, 15 generations took forever to run.

### Statistics for run 3 (n_gen = 12)
<img src='run3.png'>

The maximum score attained during the run (about 45000) was much higher than any previously seen. However, the average and best scores definitely declined even after 12 generations. So, we decided to try 8 generations repeatedly, in hopes of getting high performance while not sacrificing large amounts of time.

### Statistics for run 4 (n_gen = 8)
<img src='run4.png'>

These results were much more satisfactory. The best individual had a respectable score, and the average score rose steadily. The training time was also reasonable. This is the run we turned in.

# References
**Python Tetris game** : https://gist.github.com/silvasur/565419/7e044a90eb97eb67d600b2fb776000ba36f6fcc9

**Heuristics source** : https://codemyroad.wordpress.com/2013/04/14/tetris-ai-the-near-perfect-player/